# Trabalho de Graduação

https://www.kaggle.com/datasets/chitwanmanchanda/fraudulent-transactions-data?resource=download

**Dicionário de dados:**

step - mapeia uma unidade de tempo no mundo real. Neste caso, 1 passo é 1 hora de tempo. Total de etapas 744 (simulação de 30 dias).

type - CASH-IN, CASH-OUT, DÉBITO, PAGAMENTO e TRANSFERÊNCIA.

amount - valor da transação em moeda local.

nameOrig - cliente que iniciou a transação

oldbalanceOrg - saldo inicial antes da transação

newbalanceOrig - novo saldo após a transação

nameDest - cliente que é o destinatário da transação

oldbalanceDest - destinatário do saldo inicial antes da transação. Observe que não há informações para clientes que começam com M (Comerciantes).

newbalanceDest - novo destinatário do saldo após a transação. Observe que não há informações para clientes que começam com M (Comerciantes).

isFraud - São as transações feitas pelos agentes fraudulentos dentro da simulação. Neste conjunto de dados específico, o comportamento fraudulento dos agentes visa lucrar ao assumir o controle das contas dos clientes e tentar esvaziar os fundos transferindo para outra conta e depois sacando do sistema.

isFlaggedFraud - O modelo de negócios visa controlar transferências massivas de uma conta para outra e sinaliza tentativas ilegais. Uma tentativa ilegal neste conjunto de dados é uma tentativa de transferir mais de 200.000 em uma única transação.

### 1. Dataset

In [ ]:
# Instalação das bibliotecas
!pip install pyspark;

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import metrics
from pyspark import SparkFiles
from pyspark.sql import SparkSession
from google.colab import drive
from sklearn.tree import DecisionTreeClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 199 kB 47.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=5b4b7c8f24ff93a05d0282d14cf8fe3b48fb8c14830baa1274ba8596642b4442
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
# Conexão  com ambiente de nuvem
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Leitura da base de dados
spark = SparkSession.builder.getOrCreate()
dataframe = spark.read.csv(SparkFiles.get("/content/drive/MyDrive/TG/data.csv"), header=True, inferSchema=True, sep=",")

In [ ]:
dataframe.createOrReplaceTempView("base")

In [ ]:
query = """
  SELECT *
  FROM base
"""

spark.sql(query).show()

+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|    181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT| 11668.14|C2048537720|      41554.0|      29885.86|M123070170

## Data Cleaning

In [ ]:
# Análise e limpeza de dados
query= """
      Select isFraud, count(isFraud)
      from base
      group by isFraud
"""

spark.sql(query).show()

+-------+--------------+
|isFraud|count(isFraud)|
+-------+--------------+
|      1|          8213|
|      0|       6354407|
+-------+--------------+



In [ ]:
query = """
  SELECT CASE WHEN type = 'PAYMENT ' THEN 0 WHEN type = 'CASH-IN ' THEN 1 WHEN type = 'CASH-OUT ' THEN 2 WHEN type = 'TRANSFER' THEN 3 ELSE 4 END as type,
 amount, oldbalanceOrg, newbalanceOrig, oldbalanceDest,
  newbalanceDest, isFraud
  FROM base
  """

dataframe = spark.sql(query)
dataframe.createOrReplaceTempView('base1')

In [ ]:
query = """
        Select * from ((
          Select * from base1 WHERE isFraud=0 LIMIT 8213)
          UNION ALL (
            SELECT * from base1 WHERE isFraud=1 Limit 8213))
"""

dataframe= spark.sql(query)

In [ ]:
df = dataframe.toPandas()

# sample- amostragem aleatória
df = df.sample(frac=0.7)

In [ ]:
# Treinamento do modelo
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
5457,4,5830.20,16232.00,10401.80,0.00,0.00,0
3982,4,344106.28,1285034.84,1629141.12,3496462.74,2943614.60,0
1133,4,1668.11,1080.00,0.00,17145.04,9672.67,0
166,4,114712.48,0.00,0.00,145400.00,0.00,0
13365,4,1424002.70,1424002.70,0.00,11407.50,1435410.19,1


In [ ]:
X = df.drop(columns=['isFraud'])
y = df['isFraud']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

In [ ]:
X_train.shape, X_test.shape

((3449, 6), (8049, 6))

In [ ]:
y_train.shape, y_test.shape

((3449,), (8049,))

In [ ]:

classificador = DecisionTreeClassifier()
classificador.fit(X_train, y_train)

DecisionTreeClassifier()

In [ ]:
resultado = classificador.predict(X_test)

In [ ]:
resultado

array([0, 1, 0, ..., 1, 1, 1], dtype=int32)

In [ ]:
y_test.values

array([0, 1, 0, ..., 1, 1, 1], dtype=int32)

In [ ]:
# Avaliação do modelo
print(metrics.classification_report(y_test, resultado))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3989
           1       0.99      0.99      0.99      4060

    accuracy                           0.99      8049
   macro avg       0.99      0.99      0.99      8049
weighted avg       0.99      0.99      0.99      8049



In [ ]:
classificador.fit(X_train, y_train)

In [ ]:
# Comparação dos resultados
confusion_matrix(y_test, resultado)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, resultado).ravel()
print(tn, fp, fn, tp)

In [ ]:
acuracia = (tp + tn)/ (tp + tn + fp + fn)
print(round(acuracia, 2))

In [ ]:
# Impacto das características no aprendizado
classificador.feature_importances_

In [ ]:
# Relevância das colunas
features = pd.Series(classificador.feature_importances_, index=X_train.columns).sort_values(ascending=False)

In [ ]:
features

In [ ]:
features.plot.barh()
plt.ylabel('Features')
plt.xlabel('Importância')
plt.show();